In [2]:
from lib.data.data import Data
import numpy as np
import gc
from lib.auxiliares.esReal import es_real
from lib.volatilidades.volatilities import *
from lib.volatilidades.rolling_forecast import *

In [3]:
# Configuración de parámetros
indexes = ['SAN.MC']
input_method = 'csv'
start_get_data = '2021-07-30'
end_get_data = '2024-07-30'
start_calculation_date = '2024-06-30'
end_calculation_date = '2024-07-30'
confidence_level = 0.975
horizons = [1, 10]

In [4]:
# Inicialización del diccionario
index_dict = {item: {} for item in indexes}

In [5]:
# Procesamiento de datos y cálculo inicial
for index in index_dict:
    input_data = Data(index, start_get_data, end_get_data, input_method)
    df = input_data.data
    df['Log Returns'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
    df.dropna(inplace=True)
    index_dict[index]['Data'] = df
    index_dict[index]['ES Real'] = es_real(df, confidence_level, start_calculation_date, end_calculation_date)
    index_dict[index]['Volatilities'] = calculate_volatilities(df)

In [ ]:
# Forecasting
for index in index_dict:
    forecast_dict = {}
    for vol, vol_data in index_dict[index]['Volatilities'].items():
        forecast_dict[vol] = {}
        for horizon in horizons:
            forecast_data = roll_perceptron_forecast(vol_data, start_calculation_date, end_calculation_date, horizon)
            forecast_dict[vol][horizon] = {'PERCEPTRON': forecast_data}
            del forecast_data
            gc.collect()
            forecast_data = roll_lstm_forecast(vol_data, start_calculation_date, end_calculation_date, horizon)
            forecast_dict[vol][horizon] = {'LSTM': forecast_data}
            del forecast_data
            gc.collect()
            forecast_data = roll_random_forest_forecast(vol_data, start_calculation_date, end_calculation_date, horizon)
            forecast_dict[vol][horizon] = {'RANDOM_FOREST': forecast_data}
            del forecast_data
            gc.collect()
            
    index_dict[index]['Forecast'] = forecast_dict

Calculando 2 de 22 fechas - Progreso: 4.55%00%

In [11]:
index_dict['SAN.MC']['Forecast']['EWMA'][10]

{'PERCEPTRON':             VOLATILITY
 Date                  
 2024-07-01    0.016268
 2024-07-02    0.015625
 2024-07-03    0.013971
 2024-07-04    0.013628
 2024-07-05    0.013519
 2024-07-08    0.013124
 2024-07-09    0.012984
 2024-07-10    0.012782
 2024-07-11    0.013802
 2024-07-12    0.013351
 2024-07-15    0.014742
 2024-07-16    0.016268
 2024-07-17    0.016803
 2024-07-18    0.017761
 2024-07-19    0.018383
 2024-07-22    0.019107
 2024-07-23    0.017175
 2024-07-24    0.017627
 2024-07-25    0.016963
 2024-07-26    0.015913
 2024-07-29    0.015365
 2024-07-30    0.014693}